In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Menggunakan kamera sebagai sumber video
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Tidak dapat membuka kamera.")
    exit()

# Parameter untuk deteksi sudut (Shi-Tomasi)
feature_params = dict(maxCorners=200,
                      qualityLevel=0.3,
                      minDistance=7,
                      blockSize=7)

# Parameter untuk optical flow Lucas-Kanade
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS |
                           cv2.TERM_CRITERIA_COUNT,
                           10, 0.03))

# Warna acak untuk setiap sudut
color = np.random.randint(0, 255, (500, 3))

# Ambil frame pertama
ret, old_frame = cap.read()
if not ret:
    print("Gagal membaca frame pertama.")
    cap.release()
    exit()

old_gray = cv2.cvtColor(old_frame, 
                        cv2.COLOR_BGR2GRAY)

# Shi-Tomasi Corner Detection
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, 
                             **feature_params)

# Membuat mask image untuk menggambar jejak
mask = np.zeros_like(old_frame)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Gagal membaca frame, keluar loop.")
        break

    frame_gray = cv2.cvtColor(frame, 
                              cv2.COLOR_BGR2GRAY)

    if p0 is not None:
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, 
                                               frame_gray, 
                                               p0, None, 
                                               **lk_params)

        if p1 is not None and st is not None:
            # Hanya ambil titik yang berhasil ditrack
            good_new = p1[st == 1]
            good_old = p0[st == 1]

            # Gambar lintasan
            for i, (new, old) in enumerate(zip(good_new, 
                                               good_old)):
                a, b = new.ravel()
                c, d = old.ravel()
                mask = cv2.line(mask, (int(a), int(b)), 
                                (int(c), int(d)),
                                color[i % len(color)].
                                tolist(), 2)
                frame = cv2.circle(frame, (int(a), int(b)), 5,
                                   color[i % len(color)].
                                   tolist(), -1)

            # Update untuk iterasi berikutnya
            old_gray = frame_gray.copy()
            p0 = good_new.reshape(-1, 1, 2)

    # Gabungkan frame asli + mask
    img = cv2.add(frame, mask)

    # Convert BGR → RGB untuk matplotlib
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Tampilkan dengan matplotlib (update realtime)
    clear_output(wait=True)
    plt.imshow(img_rgb)
    plt.axis("off")
    plt.show()

    # Untuk keluar dari loop, hentikan kernel / tekan stop notebook

KeyboardInterrupt: 